In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math

## Looking at RateBeer

In [7]:
#out of exploration out of src
beers = pd.read_csv('../../baseData/RateBeer/beers.csv')

print(beers.shape)
beers.drop_duplicates(inplace=True)
print(beers.columns)
print(beers.shape)
beers.sample(10)

(442081, 14)
Index(['beer_id', 'beer_name', 'brewery_id', 'brewery_name', 'style',
       'nbr_ratings', 'overall_score', 'style_score', 'avg', 'abv',
       'avg_computed', 'zscore', 'nbr_matched_valid_ratings',
       'avg_matched_valid_ratings'],
      dtype='object')
(442081, 14)


,beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,overall_score,style_score,avg,abv,avg_computed,zscore,nbr_matched_valid_ratings,avg_matched_valid_ratings
77113,38849,Burgdorfer Weizen,2415,Burgdorfer Gasthausbrauerei,German Hefeweizen,15,50.0,84.0,3.14,5.1,3.193333,NaN,0,NaN
383974,21327,Phoenix Old Oak Premium Bitter,2676,Phoenix &#40;Oak Brewing Co.&#41;,Bitter,7,NaN,NaN,2.94,4.5,3.100000,NaN,0,NaN
7601,190802,Benelux Milton,6830,Benelux Brasserie Artisanale,Bitter,10,76.0,99.0,3.33,3.9,3.480000,NaN,0,NaN
272765,172968,Broadway Alt,10897,Broadway Brewery,Altbier,7,NaN,NaN,3.05,5.0,3.314286,NaN,0,NaN
223923,426913,De Steeg Forbidden Cherry,16118,De Steeg Brewing,Fruit Beer,2,NaN,NaN,3.02,5.7,3.200000,NaN,0,NaN
124102,501138,Berwolf Pale Ale,30831,Berwolf Brewery,American Pale Ale,1,NaN,NaN,3.04,5.0,3.500000,NaN,0,NaN
330553,226713,Brewery Vivant Wood Aged Triumph,11121,Brewery Vivant,India Pale Ale (IPA),2,NaN,NaN,2.96,4.6,3.600000,NaN,0,NaN
210030,95519,Brooklyn Savoir Faire,90,Brooklyn Brewery,Bière de Garde,61,87.0,86.0,3.36,8.0,3.432787,0.209972,61,3.432787
384847,27010,Boggart Sun Dial,3741,Boggart Hole Clough,Golden Ale/Blond Ale,18,18.0,14.0,2.59,4.7,2.533333,-1.189750,18,2.533333
392616,418873,Beavertown Lupuloid IPA Series - Armillaria Mater,14320,Beavertown Brewery,India Pale Ale (IPA),11,89.0,82.0,3.51,5.7,3.709091,NaN,0,NaN


In [8]:
#out of exploration out of src
breweries = pd.read_csv('../../baseData/RateBeer/breweries.csv')

print(breweries.shape)
breweries.drop_duplicates(inplace=True)
print(breweries.columns)
print(breweries.shape)
breweries.sample(10)

(24189, 4)
Index(['id', 'location', 'name', 'nbr_beers'], dtype='object')
(24189, 4)


,id,location,name,nbr_beers
9809,22668,Brazil,Jeffrey,1
4439,24927,Germany,EDEKA-Verbund &#40;EUCO GmbH & Netto Discounte...,47
18639,16338,"United States, Ohio",Yellow Springs Brewery,71
4480,777,Germany,Augustiner-Bräu,10
12205,16560,Sweden,Sad Robot Brewing,52
23606,15199,France,Leader Price Holding,7
18682,24208,"United States, Michigan",Barn Brewers Brewery,20
4212,18207,Germany,Kornkammer-Bräu,4
11630,24395,Slovenia,Pivovarna Stara Hruka,1
15096,19543,"United States, New York",Callicoon Brewing Company,4


In [9]:
#out of exploration out of src
users = pd.read_csv('../../baseData/RateBeer/users.csv')

print(users.shape)
users.drop_duplicates(inplace=True)
print(users.columns)
print(users.shape)
users.sample(10)

(70174, 5)
Index(['nbr_ratings', 'user_id', 'user_name', 'joined', 'location'], dtype='object')
(70174, 5)


,nbr_ratings,user_id,user_name,joined,location
58476,1,390253,bkb317,1.446808e+09,NaN
9002,1,169530,cheezwizard,1.334484e+09,NaN
9867,1,143523,pressog,1.322737e+09,NaN
18212,12,356957,AlbusDumbledor,1.423307e+09,Italy
28317,26,217765,crazymorton,1.347962e+09,Australia
69656,1,327965,nelsonsaigon,1.406369e+09,Vietnam
68804,1,76030,aperritano,1.212228e+09,"United States, California"
53540,1,197703,TexasTim,1.340878e+09,NaN
23178,4,215015,minear,1.346666e+09,South Korea
47882,2,175082,Chevitz81,1.337162e+09,"United States, South Dakota"


In [10]:
#there are 121 million lines of text... thats a lot
def parse_ratings_file(file_path, limit):
    data = []
    current_block = []
    i = 0

    with open(file_path, 'r') as file:  
        for line in file:
            i += 1
            
            if i%1_000_000 == 0: print('line x million: ', i/1_000_000)
            if i > limit :
                print('Saved first ' + str(limit/1_000_000) + ' million lines')
                break
            
            line = line.strip()  # Remove leading/trailing whitespace
            
            if not line: #if empty line, '' treated as false
                data.append(current_block)
                current_block = []
                continue #next iteration of loop
            
            if ': ' in line:  
                key, value = line.split(': ', 1)  # Split by the first occurrence of ': '
                current_block.append(value)
            

    # Convert the list of lists to a DataFrame
    return pd.DataFrame(data)


ratings_df = parse_ratings_file('../../baseData/BeerAdvocate/ratings.txt', limit = 10_000_000)

#since our txt file is in a predicatble format we can make things easy on ourselves
new_column_names = [
    'beer_name',
    'beer_id',
    'brewery_name',  
    'brewery_id',    
    'style',         
    'abv',           
    'date',          
    'user_name',     
    'user_id',       
    'appearance',    
    'aroma',         
    'palate',        
    'taste',         
    'overall',       
    'rating',        
    'text',
    'review'
]

ratings_df.columns =  new_column_names
# Display the first few rows of the DataFrame
ratings_df.sample(10)


line x million:  1.0
line x million:  2.0
line x million:  3.0
line x million:  4.0
line x million:  5.0
line x million:  6.0
line x million:  7.0
line x million:  8.0
line x million:  9.0
line x million:  10.0
Saved first 10.0 million lines


,beer_name,beer_id,brewery_name,brewery_id,style,abv,date,user_name,user_id,appearance,aroma,palate,taste,overall,rating,text,review
13649,Irish Whiskey Aged Scottish Stout,77595,Innis & Gunn,10272,English Stout,7.4,1427367600,DeadeyeOTDO,deadeyeotdo.850530,3.5,3.5,3.5,3.5,3.5,3.50,nan,False
352781,Fat Tug IPA,62911,Driftwood Brewery,18858,American IPA,7.0,1452596400,HopsForBrains,hopsforbrains.725766,4.5,4.5,4.5,4.5,4.5,4.50,nan,False
12699,Innis And Gunn Original Oak Aged Beer,12719,Innis & Gunn,10272,Scottish Ale,6.6,1097402400,cypressbob,cypressbob.3708,4.0,4.0,4.0,4.5,4.0,4.20,"330ml bottledPours with a light amber body, sl...",True
398035,Flying Monkeys The Chocolate Manifesto,104121,Flying Monkeys Craft Brewery,10796,Milk / Sweet Stout,10.0,1413108000,Devon_,devon_.826331,nan,nan,nan,nan,nan,5.00,nan,False
8072,Old Engine Oil Engineer's Reserve Blackest Ale,77058,Harviestoun Brewery Ltd.,323,English Porter,9.0,1479380400,cdub70085,cdub70085.501418,5.0,4.25,4.75,4.75,4.5,4.60,nan,False
353681,Old Cellar Dweller,46815,Driftwood Brewery,18858,American Barleywine,11.6,1421838000,Beervana,beervana.794551,4.0,4.25,4.25,4.25,4.25,4.24,nan,False
535881,Beer Geek Brunch Weasel,46987,Mikkeller ApS,13307,American Double / Imperial Stout,10.9,1446462000,kyleryan,kyleryan.954781,4.25,4.25,4.25,4.5,4.25,4.35,nan,False
538349,Beer Geek Cocoa Shake,133285,Mikkeller ApS,13307,Oatmeal Stout,12.12,1449918000,aceseth13,aceseth13.1029941,4.0,4.0,4.75,3.75,4.0,3.98,nan,False
111450,Wells Banana Bread Beer,5488,Wells & Young's Ltd,664,Fruit / Vegetable Beer,5.2,1416394800,harryhops,harryhops.893347,nan,nan,nan,nan,nan,3.60,nan,False
472038,Asahi Black (Kuronama),3969,Asahi Breweries Ltd,716,Munich Dunkel Lager,5.0,1340618400,SocalKicks,socalkicks.663388,nan,nan,nan,nan,nan,3.75,nan,False
